# imports

In [222]:

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
import os

from azureml.core import Workspace,Experiment, Environment




# Defining utility function

In [180]:
def getOrCreateCompute(ws:Workspace):
    
    from azureml.core.compute import AmlCompute
    from azureml.core.compute import ComputeTarget
    from azureml.core.compute_target import ComputeTargetException

    aml_compute_target = "testcot"
    try:
        aml_compute = AmlCompute(ws, aml_compute_target)
        print("found existing compute target.")
    except ComputeTargetException:
        print("creating new compute target")

        provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                    min_nodes = 0, 
                                                                    max_nodes = 4)    
        aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
        aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    finally:
        return aml_compute

In [181]:
def createRunConfig(ws):
    
    from azureml.core.runconfig import RunConfiguration
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    from azureml.core import Environment
    # create a new runconfig object
    run_config = RunConfiguration()
    env = Environment.get(workspace=ws, name='TITANIC', version='1')

    run_config.environment=env

    

    return run_config

# Build dev pipeline

In [182]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


# Default datastore (Azure blob storage)
def_blob_store = ws.get_default_datastore()
#def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))
#Upload file to datastore

# Use a CSV to read in the data set.
file_name = "../data/rawdata/train.csv"

if not os.path.exists(file_name):
    raise Exception(
        'Could not find CSV dataset at "%s". '
        % file_name
    )  # NOQA: E50
# Upload file to default datastore in workspace

mlops-aml-ws
mlops-rg
westeurope
f90533aa-280d-40b9-9949-a7ba0ee9511f
Blobstore's name: workspaceblobstore


# Uploading data to blob storage

In [183]:
target_path = "training-data/"
def_blob_store.upload_files(
    files=[file_name],
    target_path=target_path,
    overwrite=True,
    show_progress=False,
)

blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="test_data",
    path_on_datastore="training-data/train.csv")


aml_compute = getOrCreateCompute(ws)
run_config = createRunConfig(ws)
#processed_data1 = PipelineData("processed_data1",datastore=def_blob_store)
train_data = PipelineData("train_data1",datastore=def_blob_store)

found existing compute target.


## Preprocess step

In [184]:
source_directory="../src/preprocess/"
preprocess_step = PythonScriptStep(
    script_name="preprocess.py", 
    arguments=["--data", blob_input_data],
    inputs=[blob_input_data],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config,
    allow_reuse=True
)

In [185]:
from azureml.pipeline.core import Pipeline
Pipeline(ws, [preprocess_step])

## Training step

In [186]:
source_directory="../src/train/"
train_step = PythonScriptStep(
    name="Train Model",
    script_name="train.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    outputs=[train_data],
    arguments=[
        "--model",
        train_data
    ],
    runconfig=run_config,
    allow_reuse=True,
)

## Validation step

In [187]:
source_directory="../src/evaluation/"
evaluate_step = PythonScriptStep(
    name="Evaluate Model ",
    script_name="eval.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    inputs=[train_data],
    arguments=[
        "--model_path",
        train_data,
    ],
    runconfig=run_config,
    allow_reuse=True,
)

# Pipeline creation

In [188]:
from azureml.pipeline.core import Pipeline
train_step.run_after(preprocess_step)
evaluate_step.run_after(train_step)
steps = [preprocess_step,train_step, evaluate_step]

train_pipeline = Pipeline(workspace=ws, steps=steps)
#train_pipeline._set_experiment_name


In [189]:
pipeline_run1 = Experiment(ws, 'titanic-pipeline').submit(train_pipeline)

Created step preprocess.py [b703d092][c2104953-5cc0-4024-aeb5-d193a042ee4c], (This step is eligible to reuse a previous run's output)
Created step Train Model [bab87395][003f6e18-1c99-4255-9809-8771dbd67ce5], (This step is eligible to reuse a previous run's output)
Created step Evaluate Model  [4210397a][cc8ead9b-6605-4bb0-81e6-30f1fdb35fec], (This step will run and generate new outputs)
Using data reference test_data for StepId [91e38c09][ee17b8e7-aad7-4670-a604-b0b5a122ca4f], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun fbec2f23-da58-4a21-a3f9-88adf10b459d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fbec2f23-da58-4a21-a3f9-88adf10b459d?wsid=/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourcegroups/mlops-rg/workspaces/mlops-aml-ws&tid=e99647dc-1b08-454a-bf8c-699181b389ab


In [190]:
pipeline_run1.wait_for_completion(show_output=True)

PipelineRunId: fbec2f23-da58-4a21-a3f9-88adf10b459d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fbec2f23-da58-4a21-a3f9-88adf10b459d?wsid=/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourcegroups/mlops-rg/workspaces/mlops-aml-ws&tid=e99647dc-1b08-454a-bf8c-699181b389ab
PipelineRun Status: NotStarted
PipelineRun Status: Running


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment






PipelineRun Execution Summary
PipelineRun Status: Finished

Warnings:
Failed to post metric due to validation failure. Metric column found with a malformed type
{'runId': 'fbec2f23-da58-4a21-a3f9-88adf10b459d', 'status': 'Completed', 'startTimeUtc': '2022-12-14T15:59:22.22467Z', 'endTimeUtc': '2022-12-14T16:02:50.156161Z', 'services': {}, 'warnings': [{'source': 'RunHistoryService', 'message': 'Failed to post metric due to validation failure. Metric column found with a malformed type'}], 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2022-12-14T15:59:22.6768937+00:00","EndTime":"2022-12-14T16:02:50.0698233+00:00","Status":"Finished"}}'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles

'Finished'

In [191]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [192]:
# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
  name=local_endpoint_name, description="this is a sample local endpoint"
)

In [193]:
from azureml.core.model import Model as AMLModel
def get_model(
    model_name: str,
    model_version: int = None,  # If none, return latest model
    tag_name: str = None,
    tag_value: str = None,
    aml_workspace: Workspace = None
) -> AMLModel:
    """
    Retrieves and returns a model from the workspace by its name
    and (optional) tag.

    Parameters:
    aml_workspace (Workspace): aml.core Workspace that the model lives.
    model_name (str): name of the model we are looking for
    (optional) model_version (str): model version. Latest if not provided.
    (optional) tag (str): the tag value & name the model was registered under.

    Return:
    A single aml model from the workspace that matches the name and tag, or
    None.
    """
    if aml_workspace is None:
        print("No workspace defined - using current experiment workspace.")
        raise ValueError(
                "No workspace provided"
            )

    tags = None
    if tag_name is not None or tag_value is not None:
        # Both a name and value must be specified to use tags.
        if tag_name is None or tag_value is None:
            raise ValueError(
                "model_tag_name and model_tag_value should both be supplied"
                + "or excluded"  # NOQA: E501
            )
        tags = [[tag_name, tag_value]]

    model = None
    if model_version is not None:
        # TODO(tcare): Finding a specific version currently expects exceptions
        # to propagate in the case we can't find the model. This call may
        # result in a WebserviceException that may or may not be due to the
        # model not existing.
        model = AMLModel(
            aml_workspace,
            name=model_name,
            version=model_version,
            tags=tags)
    else:
        models = AMLModel.list(
            aml_workspace, name=model_name, tags=tags, latest=True)
        if len(models) == 1:
            model = models[0]
        elif len(models) > 1:
            raise Exception("Expected only one model")

    return model

In [ ]:
env = Environment("env",
    conda_file="../src/train/models/trained_model/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
)
model = Model(path="../src/train/models/trained_model/model.pkl")

ml_client= MLClient.from_config(DefaultAzureCredential())

In [235]:
model=get_model("titanic-xgb.pkl",aml_workspace=ws)
env = Environment.get(workspace=ws, name='TITANIC', version='3')

# Model deploying

In [236]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='../src/scoring/score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True).get_logs()

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.
/tmp/ipykernel_13312/4040477408.py:11: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-14 19:10:38+00:00 Creating Container Registry if not exists.
2022-12-14 19:10:38+00:00 Registering the environment.
2022-12-14 19:10:39+00:00 Use the existing image.
2022-12-14 19:10:39+00:00 Generating deployment configuration.
2022-12-14 19:10:41+00:00 Submitting deployment to compute.
2022-12-14 19:10:44+00:00 Checking the status of deployment my-custom-env-service..
2022-12-14 19:38:14+00:00 Checking the status of inference endpoint my-custom-env-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 605f8e37-8720-40b8-93b3-2da521e9c57b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: my-custom-env-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cotuamlcr.azurecr.io/azureml/azureml_3e60105f08daa192671f86fabaccb6ad locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more info

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 605f8e37-8720-40b8-93b3-2da521e9c57b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: my-custom-env-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cotuamlcr.azurecr.io/azureml/azureml_3e60105f08daa192671f86fabaccb6ad locally. Please refer to https://

In [ ]:
import json
import numpy

input_sample = numpy.array([
    [3,'male',22.0,1,0,7.25,'S'],
    [1,'female',38.0,1,0,71.2833,'C']])
input_payload = json.dumps({
    'data': input_sample,
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
service.delete()